# Creating SARS-CoV-2 RBD ACE2 Benchmarking Dataset

In this study we benchmarked RBD and ACE2 point mutations by using structure based ∆∆G predictors on deep mutagenesis dataset. 

These predictors are 
- HADDOCK, 
- FoldX, 
- EvoEF1, 
- MutaBind2, and
- SSIPe.    

In [1]:
import pathlib
import pandas as pd
import numpy as np

In [2]:
# defining root and data directory path 
rootdir = pathlib.Path('.').resolve(strict=True)
datadir = rootdir.parents [1] / 'files/input_files'
outputdir = rootdir.parents [1] / 'files/output_files'

## Experimental datasets

ACE2 and RBD proteins scanned with point mutations by using deep mutagenesis technique. And binding and expression values corresponding to the mutation were released. We selected only interface mutations from the result of this experiment. So our experimental dataset contains totally 263 cases - 179 ACE2 and 84 RBD cases.

In [3]:
# import dataset
ACE2 = pd.read_csv(datadir / 'ACE2_Experimental_dataset.csv', delimiter=',')
ACE2.columns=['#case_id','exp_binding']
#create protein column to represent which case belong which protein 
ACE2['protein'] = "ACE2"

RBD = pd.read_csv(datadir / 'RBD_Experimental_dataset.csv', delimiter=',')
RBD = RBD[['#case_id', 'RBD_bind_avg']]
RBD.columns=['#case_id','exp_binding']
RBD['protein'] = "RBD"

# concatanate ACE2 and RBD dataset to build experimental dataset of our study
Experimental_dataset = pd.concat([ACE2,RBD])
# setting case id as an index is necessary for joining datasets in an accurate order to the next level
Experimental_dataset=Experimental_dataset.set_index('#case_id')

## ∆∆G calculation of predictors

In our case ∆∆G represents the energy differences of the protein complex of ACE2-RBD upon a point mutation. Therefore, we assume that predicted ∆∆G must be correlated with the experimental binding energy. Since HADDOCK, FoldX and EvoEF1 do not output a ∆∆G for a mutation, we calculated the ∆∆G by subtracting ∆G of wild type from ∆G of mutant. MutaBind2 and SSIPe directly output a ∆∆G in their result pages. 

### HADDOCK

We used following formula to calculate ∆∆G HADDOCK scores:
- ∆∆G = [Mutant HADDOCK score] - [Wild type HADDOCK score]

In [4]:
# import HADDOCK score dataset
HADDOCK_score = pd.read_csv(datadir / 'HADDOCK_scores.csv', delimiter=',')

# define wild type HADDOCK score
wt = HADDOCK_score[HADDOCK_score['#case_id']=="WT"]
wt_score = wt['haddock-score'].values.tolist()

# to remove wild type row from dataset; first define case id column as an index, then drop the row via index. 
# further, reset indexing and organize column order.
HADDOCK_score = HADDOCK_score.set_index("#case_id")
HADDOCK_score = HADDOCK_score.drop(labels=['WT'], axis=0)
HADDOCK_score = HADDOCK_score.reset_index()
HADDOCK_score = HADDOCK_score[['#case_id', 'haddock-score']]

# calculate ∆∆G
HADDOCK_score['haddock-ddg'] = HADDOCK_score['haddock-score'] - wt_score

# remove the haddock-score column
HADDOCK_scores = HADDOCK_score.drop(labels=['haddock-score'], axis=1)
HADDOCK_scores=HADDOCK_scores.set_index('#case_id')

### FoldX

FoldX produces wild type score for each mutation. Therefore, we used each wild type score of corresponding mutation to calculate ∆∆G. We used following formula to calculate ∆∆G FoldX scores:

- ∆∆G = [Mutant FoldX score] - [Wild type FoldX score]

In [5]:
#import FoldX score dataset
FoldX_score = pd.read_csv(datadir / 'FoldX_scores.csv', delimiter=',')

# calculate ∆∆G
FoldX_score['foldx-ddg'] = FoldX_score['foldx-score-mut'] - FoldX_score['foldx-score-wt']

# remove foldx-score-mut and foldx-score-wt column
FoldX_scores = FoldX_score.drop(labels=['foldx-score-mut', 'foldx-score-wt'], axis=1)
FoldX_scores=FoldX_scores.set_index('#case_id')

### FoldXwater

FoldX serves as an option to contribute to crystallographic water bridges. We builded our mutations by using the Crystalwater option since the FoldX team suggested using this option in the last COVID19 related [paper](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1008450). The ∆∆G calculation is the exactly same with the FoldX.  

In [6]:
#import FoldXwater score dataset
FoldXwater_score = pd.read_csv(datadir / 'FoldXwater_scores.csv', delimiter=',')

# calculate ddg
FoldXwater_score['foldxwater-ddg'] = FoldXwater_score['foldx-score-mut'] - FoldXwater_score['foldx-score-wt']

# remove foldx-score-mut and foldx-score-wt column
FoldXwater_scores = FoldXwater_score.drop(labels=['foldx-score-mut', 'foldx-score-wt'], axis=1)
FoldXwater_scores=FoldXwater_scores.set_index('#case_id')

### EvoEF1 

For the ∆∆G calculation of EvoEF1 we followed exactly the same formula. 
- ∆∆G = [Mutant EvoEF1 score] - [Wild type EvoEF1 score]

In [7]:
#import EvoEF1 score dataset
EvoEF1_score = pd.read_csv(datadir / 'EvoEF1_scores.csv', delimiter=',')

# define WT EvoEF1 score
wt= EvoEF1_score[EvoEF1_score['#case_id'].str.contains("WT")]
wt_score = wt['evoef1-score'].values.tolist()

# to remove WT row from dataset; first define mutation_type column as index, then drop the row via index. 
# further, reset indexing and organize column order.
EvoEF1_score = EvoEF1_score.set_index("#case_id")
EvoEF1_score = EvoEF1_score.drop(labels=['WT'], axis=0)
EvoEF1_score = EvoEF1_score.reset_index()
EvoEF1_score = EvoEF1_score[['#case_id', 'evoef1-score']]

# calculate ddg
EvoEF1_score['evoef1-ddg'] = EvoEF1_score['evoef1-score'] - wt_score

# remove evoef1-score column
EvoEF1_scores = EvoEF1_score.drop(labels=['evoef1-score'], axis=1)
EvoEF1_scores=EvoEF1_scores.set_index('#case_id')

### MutaBind2,  SSIPe and UEP

There is no need to calculate ∆∆G since MutaBind2, SSIPe, and UEP output directly to ∆∆G score. Basically we selected ∆∆G column from the result pages.

In [8]:
# import  MutaBind2 ∆∆G score dataset
MutaBind2_scores = pd.read_csv(datadir / 'MutaBind2_scores.csv', delimiter=',')
# select case id and ∆∆G columns, rename the columns
MutaBind2_scores = MutaBind2_scores[['Mutation','DDG']]
MutaBind2_scores.rename(columns = {'Mutation': '#case_id', 'DDG': 'mutabind2-ddg'}, inplace = True)
MutaBind2_scores=MutaBind2_scores.set_index('#case_id')

In [9]:
# import  SSIPe ∆∆G score dataset
SSIPe_scores = pd.read_csv(datadir / 'SSIPe_result.txt', delimiter=' ')
# select case id and ∆∆G columns, rename the columns
SSIPe_scores = SSIPe_scores[['mutations','DDG']]
SSIPe_scores.rename(columns = {'mutations': '#case_id','DDG': 'ssipe-ddg'}, inplace = True)

#change the case id format FE490K -> F490K -remove chain name, E
case_id = SSIPe_scores['#case_id']

#remove ; sign
case_id_list = []
for i in case_id:
    caseid = i[:-1]
    case_id_list.append(caseid)

case_id_column = []
for i in case_id_list:
    caseid = i[0] + i[2:]
    case_id_column.append(caseid)
    
case_id_column = pd.DataFrame(case_id_column, columns=['#case_id'])
ddg_column = SSIPe_scores[['ssipe-ddg']]
# concatanate case id and ddg columns
SSIPe_scores = pd.concat([case_id_column,ddg_column], axis=1)
SSIPe_scores = SSIPe_scores.set_index('#case_id')

In [10]:
# import UEP ∆∆G score dataset
UEP_scores = pd.read_csv(datadir / '6m0j_UEP_A_E.csv', delimiter=',')

# UEP result page is as a 2 dimentional array, we converged the file to 1 dimentional array by using following steps 


# we used single letter amino acid abbreviation, so we converted 3 letter abbreviation to single
abbreviation = dict(ALA='A', ARG='R', ASN='N', ASP='D', CYS='C', GLN='Q', GLU='E', GLY='G', HIS='H', ILE='I',
                LEU='L', LYS='K', MET='M', PHE='F', PRO='P', SER='S', THR='T', TRP='W', TYR='Y', VAL='V') 
aa_columns=UEP_scores.columns.tolist()
x = aa_columns[1:]
single_letter_aa = []
for i in x:
    aa=abbreviation[i]
    single_letter_aa.append(aa)
    
# array format for extract case list, cases represented as [wild-type residue][position][mutant residue], A111C
UEP_array = pd.DataFrame.to_numpy(UEP_scores)
caselist = []
for case in UEP_array:
    pos_wt = case[0][2:]
    wt = pos_wt[-3:]
    wt=abbreviation[wt]
    position = pos_wt[:-4]
    wt_position = [wt+position]
    for i in wt_position:
        for e in single_letter_aa:
            caselist.append(f'{i}{e}')  

# list format for extract ∆∆G scores
UEP_list = UEP_scores.values.tolist()             
ddg_values = []
for i in UEP_list:
    a=i[1:]
    ddg_values.append(a)
list_ddg = []
for a in range(28):
    for i in ddg_values[a]:
        list_ddg.append(i)

# append case id and ∆∆G score columns        
data=[]
for i,k in zip(caselist, list_ddg):
            data.append([i,k])
UEP_score = pd.DataFrame(data, columns=['#case_id', 'uep-ddg'])
# NaN cases must be romoved from the dataset
UEP_scores=UEP_score.dropna()
UEP_scores = UEP_scores.set_index('#case_id')

## Merge Predictors' ∆∆G Together

We will have 2 dataset for further analysis, one of them main dataset consists of 263 cases. The other one UEP dataset consists of 129 cases, contains only the common cases between main dataset and UEP output. Consider that UEP suggests a ∆∆G energy of a mutation if it is highly-packed residue with has at least 2 non-covalent bond. Therefore, we missed nearly half of our cases while constructing UEP dataset. 

- Predictors' performance on main dataset presents the performance on interface mutations.
- Predictors' performance on UEP dataset presents the performances on highly-packed interface mutations. 

In [11]:
# main dataset
predictors = [Experimental_dataset, HADDOCK_scores, FoldX_scores, FoldXwater_scores, EvoEF1_scores, MutaBind2_scores, SSIPe_scores]
SARS_CoV_2_RBD_ACE2_benchmarking_dataset = pd.concat(predictors, axis=1,join="inner")

# reset the index and save the main benchmarking dataset
SARS_CoV_2_RBD_ACE2_benchmarking_dataset=SARS_CoV_2_RBD_ACE2_benchmarking_dataset.reset_index()
SARS_CoV_2_RBD_ACE2_benchmarking_dataset.to_csv(outputdir / 'SARS-CoV-2-RBD_ACE2_benchmarking_dataset.csv', index=False, float_format='%.2f')

In [12]:
# UEP dataset
predictors = [Experimental_dataset, HADDOCK_scores, FoldX_scores, FoldXwater_scores, EvoEF1_scores, MutaBind2_scores, SSIPe_scores, UEP_scores]
UEP_SARS_CoV_2_RBD_ACE2_benchmarking_dataset = pd.concat(predictors, axis=1,join="inner")

# reset the index and save the UEP benchmarking dataset 
UEP_SARS_CoV_2_RBD_ACE2_benchmarking_dataset=UEP_SARS_CoV_2_RBD_ACE2_benchmarking_dataset.reset_index()
UEP_SARS_CoV_2_RBD_ACE2_benchmarking_dataset.to_csv(outputdir / 'UEP_SARS-CoV-2-RBD_ACE2_benchmarking_dataset.csv', index=False, float_format='%.2f')